In [ ]:
import os
import datetime
import pandas as pd

path = os.path.join('.', 'history', 'bb_to_predict.xlsx')
df = pd.read_excel(path, sheet_name='Sheet1')
print(df)

path = os.path.join('.', 'history', 'predition.xlsx')
df.to_excel(path, sheet_name='Sheet1')


Empty DataFrame
Columns: [year, month, day, div, home, away]
Index: []


In [16]:
import os
import datetime
import pandas as pd
df = pd.DataFrame(
    {
        "Date and time": [
            datetime.datetime(2015, 1, 1, 11, 30, 55),
            datetime.datetime(2015, 1, 2, 1, 20, 33),
            datetime.datetime(2015, 1, 3, 11, 10),
            datetime.datetime(2015, 1, 4, 16, 45, 35),
            datetime.datetime(2015, 1, 5, 12, 10, 15),
        ],
        "Dates only": [
            datetime.date(2015, 2, 1),
            datetime.date(2015, 2, 2),
            datetime.date(2015, 2, 3),
            datetime.date(2015, 2, 4),
            datetime.date(2015, 2, 5),
        ],
    }
)

writer = pd.ExcelWriter(
    os.path.join(".", "history", "pandas_datetime.xlsx"),
    engine="xlsxwriter",
    datetime_format="mmm d yyyy hh:mm:ss",
    date_format="yyyy-mm-dd",
)

df.to_excel(writer, sheet_name="Sheet1")

workbook = writer.book
worksheet = writer.sheets["Sheet1"]

(max_row, max_col) = df.shape

worksheet.set_column(1, max_col, 20)

writer.close()

In [36]:
import pandas as pd
import os

df = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 15],
                   'temp': [11, 23]
                   })
df = pd.to_datetime(df[['year', 'day', 'month']], format="yyyy-mm-dd").dt.date
print(df)
df.to_excel(os.path.join(".", "history", "test.xlsx"), sheet_name="my sheet")

0    2015-02-04
1    2016-03-15
dtype: object


In [ ]:
import pandas as pd
import html5lib
import datetime
import locale
import pytz
import lxml as lx
import openpyxl as oxl

url = "https://www.rba.gov.au/statistics/cash-rate/"

tables = pd.read_html(url)

table = tables[0]

table['Effective Date'] = pd.to_datetime(table['Effective Date'], format='%d %b %Y', errors='coerce').dt.date
table = table.dropna()
table.to_excel('rates.xlsx')

writer = pd.ExcelWriter("rates.xlsx",
                        engine='xlsxwriter',
                        date_format='m/d/yyyy')

# Convert the dataframe to an XlsxWriter Excel object.
table.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter workbook and worksheet objects in order to set the column
# widths, to make the dates clearer.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']
worksheet.set_column('B:E', 20)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)>

In [1]:
import pandas as pd
import os

df = pd.DataFrame({'a': pd.Series(dtype='int'),
                   'b': pd.Series(dtype='str'),
                   'c': pd.Series(dtype='float'),
                   'd': pd.Series(dtype='datetime64[D]')
                   })
df.to_excel(os.path.join(".", "history", "test.xlsx"), sheet_name="my sheet")



In [ ]:
import pandas as pd

data = { 'col1': [1,1,3], 'col2': [1,1,5], 'col3': [1,2,3] }
df = pd.DataFrame(data)
print(df)
df = df.drop_duplicates(subset=['col1', 'col2'])
print(df)

   col1  col2  col3
0     1     1     1
1     2     2     2
2     3     5     3
   col1  col2  col3
0     1     1     1
1     2     2     2
2     3     5     3


In [4]:

def generator(aa):
    for a in aa:
        if aa.index(a) < 4:
            yield a
        else:
            aa = [a + 5 for a in aa]
            yield generator(aa)

bb = generator([1,2,3,4,5])

for b in bb:
    print(b)

1
2
3
4
<generator object generator at 0x00000205F78CC040>


In [2]:
d = { 1: (11, 12), 2: (22, 34) }
f = { id: value for (id, value) in d.items() if id > 1}
print(f)

{2: (22, 34)}


In [7]:
import tensorflow as tf
a = tf.constant([[1,2,3,4,5], [6,7,8,9,0]])
tf.gather(a, [0, 2,3], axis=-1)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 3, 4],
       [6, 8, 9]])>

In [ ]:
import tensorflow as tf

len = tf.constant([1,2,3], dtype=tf.int32)

total_mask = []
for s in len:
    mask = tf.concat([tf.ones(s, dtype=tf.int32), tf.zeros(5-s, dtype=tf.int32)], axis=-1)
    mask = mask[:, tf.newaxis] & mask[tf.newaxis, :]
    total_mask.append(mask)
    print(mask)
total_mask = tf.stack(total_mask, axis=0)
print(total_mask.shape)
print(total_mask)


tf.Tensor(
[[1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]], shape=(5, 5), dtype=int32)
tf.Tensor(
[[1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]], shape=(5, 5), dtype=int32)
tf.Tensor(
[[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]], shape=(5, 5), dtype=int32)
(3, 5, 5)
tf.Tensor(
[[[1 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 1 0 0 0]
  [1 1 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 1 1 0 0]
  [1 1 1 0 0]
  [1 1 1 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]], shape=(3, 5, 5), dtype=int32)


In [39]:
import pandas as pd
import numpy as np

data = { 'col1': [1, 2, 3, 4, np.nan], 'col2': [1, np.nan, 3, 4, 5], 'col3': [1, 2, np.nan, 4, np.nan] }
df = pd.DataFrame(data)

df2 = df.loc[:, ['col1', 'col3']]
print(df2)
array = df2['col1'].dropna()
print(array)

df2.loc[:, 'col1'] = -3

for index, row in df.iterrows():
    # if pd.isnull(row['col1']) or pd.isnull(row['col2']):
    #     print(row['col1'], row['col2'])
    if pd.isnull( row['col1'] ):
        row['col1'] = -5

print(df)
# df.loc[df.isnull().any(axis=1)]

   col1  col3
0   1.0   1.0
1   2.0   2.0
2   3.0   NaN
3   4.0   4.0
4   NaN   NaN
0    1.0
1    2.0
2    3.0
3    4.0
Name: col1, dtype: float64
   col1  col2  col3
0   1.0   1.0   1.0
1   2.0   NaN   2.0
2   3.0   3.0   NaN
3   4.0   4.0   4.0
4  -5.0   5.0   NaN


In [4]:
import os
COUNTRY = 'England'
THEME = 'test_theme'
CountryThemeFolderPath = "./data/football-data-co-uk/" + COUNTRY + '/' + THEME
os.makedirs(CountryThemeFolderPath, exist_ok=True)
os.makedirs(os.path.join(CountryThemeFolderPath, '_id_map'), exist_ok=True)
os.makedirs(os.path.join(CountryThemeFolderPath, '_dataaset'), exist_ok=True)
os.makedirs(os.path.join(CountryThemeFolderPath, '_checkpoint'), exist_ok=True)

In [5]:
import http.client

conn = http.client.HTTPSConnection("api.sportmonks.com")
payload = ''
headers = {}
conn.request("GET", "/v3/football/fixtures/head-to-head/{team_id_1}/{team_id2}?api_token=YOUR_TOKEN", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

gaierror: [Errno 11001] getaddrinfo failed

In [ ]:
print(None)

In [ ]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
# !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
# !pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
# !pip install protobuf~=3.20.3
# !pip install -q tensorflow_datasets
# !pip install -q -U tensorflow-text tensorflow

In [ ]:
#  no name 'resource'?
#  pip3 uninstall tensorflow_datasets; pip3 install tensorflow_datasets==4.9.2

In [ ]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import os
import data_helpers
import data_generator
from config import config

In [ ]:
df = data_helpers.read_large_excel(config['data'], "contest_total_data")
df = data_helpers.improve_contest_data(df)

In [ ]:
teams = list(set(list(df['teams_home_team_id']) + list(df['teams_away_team_id'])))
teams_string = [str(team) for team in teams]
teams_text = " ".join(teams_string)

corpus_file = "./data/tokenizers/team_ids_text.txt"
f = open(corpus_file, "w", encoding="utf-8")
f.write(teams_text)
f.close()

corpus_files = [corpus_file]
unknown_token = config['unknown_token']
special_tokens = [unknown_token] + ["[HOME]", "[AWAY]"]
vocab_size = len(teams_string) + len(special_tokens)

tokenizer_team = data_generator.createSimpleTokenizer(corpus_files, vocab_size, unknown_token, special_tokens)

In [ ]:
tokenizer_team.get_vocab_size()

In [ ]:
encoding = tokenizer_team.encode("1 2 3 what? 333 -5 4)e_t grw0 1234 [HOME] [AWAY]")
# encoding = tokenizer.encode("")
print(encoding.tokens)
print(encoding.type_ids)
print(encoding.ids)

tokenizer_team.decode(encoding.ids)

In [ ]:
tokenizer_team.decode([3, 2046, 2921, 0, 3105, 0, 0, 0, 804, 1, 2])

In [ ]:
import re

countries_string = list(set(list(df['league_country']) + list(df['home_team_country'])+ list(df['away_team_country'])))
# print(countries_string.index('ViÃ° MargÃ¡ir'))
countries_string = [re.sub(r"\s", "_", item) for item in countries_string]    # replace spaces with a '_'
# print(countries_string.index('ViÃ°_MargÃ¡ir'))
# teams_string = [str(team) for team in teams]
countries_text = " ".join(countries_string)

corpus_file = "./data/tokenizers/country_ids_text.txt"
f = open(corpus_file, "w", encoding="utf-8")
f.write(countries_text)
f.close()

corpus_files = [corpus_file]
unknown_token = config['unknown_token']
special_tokens = [unknown_token] + ["[LEAGUE_COUNTRY]", "[HOME_COUNTRY]", "[AWAY_COUNTRY]"]
vocab_size = len(countries_string) + len(special_tokens)

tokenizer_country = data_generator.createSimpleTokenizer(corpus_files, vocab_size, unknown_token, special_tokens)

In [ ]:
countries = ['Argentina', 'Italy', 'WHAT', 'France', 'USA', 'Canada', '[UNK]', 'Unknown', '[AWAY_COUNTRY]', 'Chinese Taipei', 'ViÃ° MargÃ¡ir']
countries = [re.sub(r"\s", "_", item) for item in countries]
print(countries)
countries = " ".join(countries)
encoding = tokenizer_country.encode(countries)
# encoding = tokenizer.encode("")
print(encoding.tokens)
print(encoding.type_ids)
print(encoding.ids)

tokenizer_country.decode(encoding.ids)

In [ ]:
filename = 'ids-5.0-0.01-15.0-0.9-0.8-10-0.000005-26.64-32601-15'
(a_conductance_search, b_conductance_search, conductance365, 
        conductanceMedTeam, ids_max, minConductanceStep, hours, nDataPoints, maxLen) \
= data_helpers.filename_to_dataParams(filename)
path = os.path.join('./data', filename + '.json')
fixture_id_to_ids = data_helpers.LoadJsonData(path)

In [ ]:
lengths = [len(ids) for ids in fixture_id_to_ids.values()]
plt.hist(lengths, np.linspace(0, 200, 101))
plt.ylim(plt.ylim())
maxLen = max(lengths)
plt.plot([maxLen, maxLen], plt.ylim())
plt.title(f'Max length of ids: {maxLen}')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
for elem in dataset:
    print(elem)

In [ ]:
ds

In [ ]:
pair_cols = ['teams_home_team_id', 'teams_away_team_id']
dummy_pair = "[HOME] [AWAY]"
country_cols = ['league_country', 'home_team_country', 'away_team_country']
dummy_countries = "[LEAGUE_COUNTRY] [HOME_COUNTRY] [AWAY_COUNTRY]"
odds_cols = ['winning_odds_home', 'winning_odds_draw', 'winning_odds_away']
dummy_wining_percent = (80, 10, 10)         
dummy_odds = [
    data_helpers.get_odds(dummy_wining_percent[0]/100, config['bookie_profit_percent']),
    data_helpers.get_odds(dummy_wining_percent[1]/100, config['bookie_profit_percent']),
    data_helpers.get_odds(dummy_wining_percent[2]/100, config['bookie_profit_percent']),
]
outcome_cols = ['outcome']
dummy_outcome = [1, 0, 0]


In [ ]:
def get_BB(df, game_id):
    try:
        if game_id >= 0:
            pair = list(df.loc[df['fixture_id'] == game_id, pair_cols].iloc[0])
            pair_str = [str(team) for team in pair]
            pair_text = " ".join(pair_str)
            pair_tokens = tokenizer_team.encode(pair_text).ids
            print(pair_text, pair_tokens)
            countries = list(df.loc[df['fixture_id'] == game_id, country_cols].iloc[0])
            countries = [re.sub(r"\s", "_", item) for item in countries]
            countries_text = " ".join(countries)
            countries_tokens = tokenizer_country.encode(countries_text).ids
            print(countries_text, countries_tokens)
            odds = list(df.loc[df['fixture_id'] == game_id, odds_cols].iloc[0])
            print(odds)
        elif game_id == -1:
            pair_text = dummy_pair
            pair_tokens = tokenizer_team.encode(pair_text).ids
            print(pair_text, pair_tokens)
            countries_text = dummy_countries
            countries_tokens = tokenizer_country.encode(countries_text).ids
            print(countries_text, countries_tokens)      
            odds = dummy_odds
            print(odds)

        BB = pair_tokens + countries_tokens + odds
        BB = np.array(BB)
        print(BB)
        return BB
    except:
        raise Exception("Failed to get_BB. game_id: {}".format(game_id))

def get_AB(df, game_id):
    try:
        if game_id >= 0:
            outcome = list(df.loc[df['fixture_id'] == game_id, outcome_cols].iloc[0])
            outcome = (1, 0, 0) if outcome == 0 else (0, 1, 0) if outcome == 1 else (0, 0, 1)
        elif game_id == -1:   # Hyperthetic game
            print("id=-1")
            outcome = dummy_outcome     # [HOME] team always wins.
            print(outcome)
        
        AB = outcome
        AB = np.array(AB, dtype=np.float32)
        print(AB)
        return AB
    except:
        raise Exception("Failed to get_AB. game_id: {}".format(game_id))

In [ ]:
def generator():    
    for baseId, ids in fixture_id_to_ids.items():
        print(baseId, ids)
        sequence = []
        if len(ids) > 0:
            for id in ids:
                BB = get_BB(df, id)
                AB = get_AB(df, id)
                if BB.shape[0] + AB.shape[0] > 11:
                    print("shape", baseId, ids)
                sequence.append(np.concatenate((BB, AB)))
                print("1", sequence[-1].shape)
        else:
            BB = get_BB(df, -1)
            AB = get_AB(df, -1)
            sequence.append(np.concatenate((BB, AB)))
            print("1", sequence[-1].shape)

        sequence = np.stack(sequence, axis=0)
        print("shape", sequence.shape)
        BB = get_BB(df, int(baseId))
        AB = get_AB(df, int(baseId))
        if BB.shape[0] + AB.shape[0] > 11:
            print("shape", baseId, ids)
        baseOutput = np.concatenate((BB, AB))

        totalOutput = (baseOutput, sequence)
        print("total: ", totalOutput)

        yield totalOutput


In [ ]:
ds = tf.data.Dataset.from_generator(
    generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape((11,)), tf.TensorShape((None, 11))),
    args=()
)

In [ ]:
for item in ds:
    print(item)
    # break

In [ ]:
tf.data.experimental.save(ds, "./data/dataset")

In [ ]:
ds2 = tf.data.experimental.load("./data/dataset")

In [ ]:
for item in ds2:
    print(item)

In [ ]:
generator()